## Ridge, Lasso, Elastic Net

### Problem Statement
Unemployment is a big socio-economic and political concern for any country and, hence, managing it is a chief task for any government. In this guide, we will try to build regression algorithms for predicting unemployment within an economy.

The data used in this project was produced from US economic time series data available from http://research.stlouisfed.org/fred2. The data contains 574 rows and 5 variables, as described below:

#### psavert - personal savings rate.
#### pce - personal consumption expenditures, in billions of dollars.
#### uempmed - median duration of unemployment, in weeks.
#### pop - total population, in thousands.
#### unemploy- number of unemployed in thousands (dependent variable).


### Step 1 - Loading the Required Libraries and Modules

In [67]:
import warnings
warnings.filterwarnings('ignore')

In [62]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import scale
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold,GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

### Step 2 - Reading the Data and Performing Basic Data Checks

In [40]:
import pandas as pd  # Importing pandas for data manipulation and analysis

# Reading data from an Excel file into a DataFrame
df = pd.read_excel('data.xlsx')  

# Printing the shape of the DataFrame (rows, columns) to understand its size
print(df.shape)  

# Displaying a summary of the DataFrame, including data types and non-null counts
print(df.info())  

# Generating descriptive statistics for numerical columns in the DataFrame
df.describe()  

(561, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   PCE       561 non-null    int64
 1   POP       561 non-null    int64
 2   PSAVERT   561 non-null    int64
 3   UNEMPLOY  561 non-null    int64
 4   UEMPMED   561 non-null    int64
dtypes: int64(5)
memory usage: 22.0 KB
None


,PCE,POP,PSAVERT,UNEMPLOY,UEMPMED
count,561.000000,561.000000,561.000000,561.000000,561.000000
mean,7302.024955,278991.654189,7.770053,8356.536542,9.588235
std,4529.315730,36192.001529,3.088382,2437.927997,4.129139
min,1169.000000,218440.000000,2.000000,5481.000000,2.000000
25%,3294.000000,244610.000000,6.000000,6682.000000,7.000000
50%,6543.000000,280976.000000,7.000000,7784.000000,8.000000
75%,10783.000000,312403.000000,9.000000,8937.000000,10.000000
max,18266.000000,334944.000000,34.000000,23050.000000,25.000000


## Step 3 - Creating Arrays for the Features and the Response Variable

In [42]:
# Defining the target column (dependent variable) to be used for modeling or analysis
target_column = ['UNEMPLOY']

# Creating a list of predictor (independent) columns by excluding the target column from all columns in the DataFrame
predictors = list(set(list(df.columns)) - set(target_column))

# Normalizing the predictor columns by dividing each value by the maximum value in its respective column
df[predictors] = df[predictors] / df[predictors].max()

# Generating descriptive statistics for the DataFrame after normalization
df.describe()

,PCE,POP,PSAVERT,UNEMPLOY,UEMPMED
count,561.000000,561.000000,561.000000,561.000000,561.000000
mean,0.399760,0.832950,0.228531,8356.536542,0.383529
std,0.247964,0.108054,0.090835,2437.927997,0.165166
min,0.063999,0.652169,0.058824,5481.000000,0.080000
25%,0.180335,0.730301,0.176471,6682.000000,0.280000
50%,0.358207,0.838875,0.205882,7784.000000,0.320000
75%,0.590332,0.932702,0.264706,8937.000000,0.400000
max,1.000000,1.000000,1.000000,23050.000000,1.000000


## Step 4 - Creating the Training and Test Datasets

In [43]:
# Extracting predictor (independent variable) values as a NumPy array
X = df[predictors].values

# Extracting target (dependent variable) values as a NumPy array
y = df[target_column].values

# Splitting the dataset into training and testing sets (70% training, 30% testing)
# random_state ensures reproducibility of the split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)

# Printing the shape of the training and testing sets for predictors (X)
print(X_train.shape)  # Shape of the training set (rows, columns)
print(X_test.shape)   # Shape of the testing set (rows, columns)

(392, 4)
(169, 4)


## Step 5 - Build, Predict and Evaluate the Regression Model

In [44]:
# Initializing a Linear Regression model
lr = LinearRegression()

# Training (fitting) the Linear Regression model using the training data
lr.fit(X_train, y_train)

LinearRegression()

In [45]:
# Predicting target values for the training data using the trained Linear Regression model
pred_train_lr = lr.predict(X_train)

# Calculating and printing the root mean squared error (RMSE) for the training predictions
print(np.sqrt(mean_squared_error(y_train, pred_train_lr)))

# Calculating and printing the R-squared score for the training predictions (model's performance on training data)
print(r2_score(y_train, pred_train_lr))

# Predicting target values for the testing data using the trained Linear Regression model
pred_test_lr = lr.predict(X_test)

# Calculating and printing the root mean squared error (RMSE) for the testing predictions
print(np.sqrt(mean_squared_error(y_test, pred_test_lr)))

# Calculating and printing the R-squared score for the testing predictions (model's performance on testing data)
print(r2_score(y_test, pred_test_lr))

1497.478192850254
0.6137780635501369
1380.0909818486987
0.693523637493907


## Regularized Regression

### Ridge Regression

In [46]:
# Initializing a Ridge Regression model with a regularization strength (alpha)
# Alpha is the hyperparameter that controls the degree of regularization:
# - A smaller alpha (e.g., 0.01) applies less regularization and behaves closer to standard linear regression.
# - A larger alpha increases regularization, reducing the model's complexity to prevent overfitting.
rr = Ridge(alpha=0.01)

# Training (fitting) the Ridge Regression model using the training data
rr.fit(X_train, y_train)

# Predicting target values for the training data using the trained Ridge Regression model
pred_train_rr = rr.predict(X_train)

# Calculating and printing the root mean squared error (RMSE) for the training predictions
print(np.sqrt(mean_squared_error(y_train, pred_train_rr)))

# Calculating and printing the R-squared score for the training predictions (model's performance on training data)
print(r2_score(y_train, pred_train_rr))

# Predicting target values for the testing data using the trained Ridge Regression model
pred_test_rr = rr.predict(X_test)

# Calculating and printing the root mean squared error (RMSE) for the testing predictions
print(np.sqrt(mean_squared_error(y_test, pred_test_rr)))

# Calculating and printing the R-squared score for the testing predictions (model's performance on testing data)
print(r2_score(y_test, pred_test_rr))

1498.093457657057
0.6134606264305966
1385.726009342799
0.6910157905469532


#### The above output shows that the RMSE and R-squared values for the Ridge Regression model on the training data is 1498 thousand and 61.3 percent, respectively. For the test data, the result for these metrics is 1895 thousand and 69 percent, respectively.

In [81]:
# Setting alpha as 0.0001, which is very close to 0
# - A smaller alpha value applies minimal regularization, making Ridge Regression behave similarly to standard Linear Regression.
# - This can result in a better fit for training data but may increase the risk of overfitting for test data.
rr = Ridge(alpha=0.0001)

# Training (fitting) the Ridge Regression model using the training data
rr.fit(X_train, y_train)

# Predicting target values for the training data using the trained Ridge Regression model
pred_train_rr = rr.predict(X_train)

# Calculating and printing the root mean squared error (RMSE) for the training predictions
print(np.sqrt(mean_squared_error(y_train, pred_train_rr)))

# Calculating and printing the R-squared score for the training predictions (model's performance on training data)
print(r2_score(y_train, pred_train_rr))

# Predicting target values for the testing data using the trained Ridge Regression model
pred_test_rr = rr.predict(X_test)

# Calculating and printing the root mean squared error (RMSE) for the testing predictions
print(np.sqrt(mean_squared_error(y_test, pred_test_rr)))

# Calculating and printing the R-squared score for the testing predictions (model's performance on testing data)
print(r2_score(y_test, pred_test_rr))

1497.4782632414215
0.6137780272402742
1380.142892758321
0.693500581382136


In [82]:
# Setting alpha as 0.03 for moderate regularization
# - A slightly higher alpha compared to very small values (e.g., 0.0001) introduces more regularization.
# - This helps in balancing the trade-off between overfitting (too complex) and underfitting (too simple).
rr = Ridge(alpha=0.03)

# Training (fitting) the Ridge Regression model using the training data
rr.fit(X_train, y_train)

# Predicting target values for the training data using the trained Ridge Regression model
pred_train_rr = rr.predict(X_train)

# Calculating and printing the root mean squared error (RMSE) for the training predictions
print(np.sqrt(mean_squared_error(y_train, pred_train_rr)))

# Calculating and printing the R-squared score for the training predictions (model's performance on training data)
print(r2_score(y_train, pred_train_rr))

# Predicting target values for the testing data using the trained Ridge Regression model
pred_test_rr = rr.predict(X_test)

# Calculating and printing the root mean squared error (RMSE) for the testing predictions
print(np.sqrt(mean_squared_error(y_test, pred_test_rr)))

# Calculating and printing the R-squared score for the testing predictions (model's performance on testing data)
print(r2_score(y_test, pred_test_rr))

1501.8007228969986
0.6115451557077576
1398.4722561111562
0.6853054234367043


In [83]:
# Setting alpha as 0.02 for moderate regularization
# - A slightly higher alpha compared to very small values like 0.0001 still applies regularization.
# - This helps in balancing the trade-off between overfitting and underfitting.
rr = Ridge(alpha=0.02)

# Training (fitting) the Ridge Regression model using the training data
rr.fit(X_train, y_train)

# Predicting target values for the training data using the trained Ridge Regression model
pred_train_rr = rr.predict(X_train)

# Calculating and printing the root mean squared error (RMSE) for the training predictions
print(np.sqrt(mean_squared_error(y_train, pred_train_rr)))

# Calculating and printing the R-squared score for the training predictions (model's performance on training data)
print(r2_score(y_train, pred_train_rr))

# Predicting target values for the testing data using the trained Ridge Regression model
pred_test_rr = rr.predict(X_test)

# Calculating and printing the root mean squared error (RMSE) for the testing predictions
print(np.sqrt(mean_squared_error(y_test, pred_test_rr)))

# Calculating and printing the R-squared score for the testing predictions (model's performance on testing data)
print(r2_score(y_test, pred_test_rr))

1499.6444649765567
0.6126598275950197
1391.9608149346202
0.6882291064223477


### Lasso Regression
Lasso regression, or the Least Absolute Shrinkage and Selection Operator, is also a modification of linear regression. In Lasso, the loss function is modified to minimize the complexity of the model by limiting the sum of the absolute values of the model coefficients (also called the l1-norm).

The loss function for Lasso Regression can be expressed as below:

Loss function = OLS + alpha * summation (absolute values of the magnitude of the coefficients)

In [76]:
# Initializing a Lasso Regression model with a regularization parameter (alpha)
# - Lasso (Least Absolute Shrinkage and Selection Operator) applies L1 regularization.
# - The alpha parameter controls the strength of the regularization:
#   - Smaller alpha values apply less regularization, allowing more coefficients to remain non-zero.
#   - Larger alpha values apply stronger regularization, potentially shrinking more coefficients to zero (feature selection).
model_lasso = Lasso(alpha=0.01)

# Training (fitting) the Lasso Regression model using the training data
model_lasso.fit(X_train, y_train)

# Predicting target values for the training data using the trained Lasso Regression model
pred_train_lasso = model_lasso.predict(X_train)

# Calculating and printing the root mean squared error (RMSE) for the training predictions
print(np.sqrt(mean_squared_error(y_train, pred_train_lasso)))

# Calculating and printing the R-squared score for the training predictions (model's performance on training data)
print(r2_score(y_train, pred_train_lasso))

# Predicting target values for the testing data using the trained Lasso Regression model
pred_test_lasso = model_lasso.predict(X_test)

# Calculating and printing the root mean squared error (RMSE) for the testing predictions
print(np.sqrt(mean_squared_error(y_test, pred_test_lasso)))

# Calculating and printing the R-squared score for the testing predictions (model's performance on testing data)
print(r2_score(y_test, pred_test_lasso))

1497.478370670584
0.6137779718251066
1380.177110161681
0.6934853833264865


In [77]:
# Initializing a Lasso Regression model with a higher regularization parameter (alpha=0.5)
# - A larger alpha applies stronger L1 regularization, which:
#   - Shrinks coefficients more aggressively.
#   - Forces some coefficients to become exactly zero, effectively performing feature selection.
model_lasso1 = Lasso(alpha=0.5)

# Training (fitting) the Lasso Regression model using the training data
model_lasso1.fit(X_train, y_train)

# Predicting target values for the training data using the trained Lasso Regression model
pred_train_lasso1 = model_lasso1.predict(X_train)

# Calculating and printing the root mean squared error (RMSE) for the training predictions
print(np.sqrt(mean_squared_error(y_train, pred_train_lasso1)))

# Calculating and printing the R-squared score for the training predictions (model's performance on training data)
print(r2_score(y_train, pred_train_lasso1))

# Predicting target values for the testing data using the trained Lasso Regression model
pred_test_lasso1 = model_lasso1.predict(X_test)

# Calculating and printing the root mean squared error (RMSE) for the testing predictions
print(np.sqrt(mean_squared_error(y_test, pred_test_lasso1)))

# Calculating and printing the R-squared score for the testing predictions (model's performance on testing data)
print(r2_score(y_test, pred_test_lasso1))

1497.9206123344165
0.6135498166852426
1384.9523443675569
0.6913607123093292


### CV-Hyper parameter tunning 

In [66]:
# Setting up cross-validation with RepeatedKFold
# - n_splits=10: Divides the data into 10 folds for cross-validation.
# - n_repeats=3: Repeats the cross-validation process 3 times for more robust results.
# - random_state=1: Ensures reproducibility of the folds.
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

# Generating a range of alpha values from 0 to 0.2 with 21 equally spaced points
# - These alpha values will be tested to find the optimal regularization strength for Lasso.
lasso_alphas = np.linspace(0, 0.2, 21)

# Initializing a Lasso Regression model (without setting alpha explicitly yet)
lasso = Lasso()

# Defining a dictionary for the hyperparameter grid
# - The key 'alpha' corresponds to the list of alpha values to be tested.
grid = dict()
grid['alpha'] = lasso_alphas

# Setting up a GridSearchCV to find the best alpha value
# - `lasso`: The Lasso Regression model.
# - `grid`: The hyperparameter grid containing alpha values.
# - `scoring='neg_mean_absolute_error'`: The performance metric to minimize.
# - `cv=cv`: The cross-validation strategy defined earlier.
# - `n_jobs=-1`: Utilizes all available CPU cores for parallel processing.
gscv = GridSearchCV(
    lasso, grid, scoring='neg_mean_absolute_error',
    cv=cv, n_jobs=-1
)

# Performing the grid search to find the best alpha value
results = gscv.fit(X, y)

# Printing the best cross-validated mean absolute error (MAE) score
print('MAE: %.5f' % results.best_score_)

# Printing the optimal alpha value found during grid search
print('Config: %s' % results.best_params_)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 561745629.6253227, tolerance: 306403.849933

MAE: -1070.68260
Config: {'alpha': 0.2}


### ElasticNet Regression
ElasticNet combines the properties of both Ridge and Lasso regression. It works by penalizing the model using both the l2-norm and the l1-norm.

In [49]:
# Elastic Net Regression model with L1 and L2 regularization
# - `alpha=0.01`: Controls the overall strength of regularization.
# - Elastic Net combines L1 (Lasso) and L2 (Ridge) penalties for regularization.
model_enet = ElasticNet(alpha=0.01)

# Training (fitting) the Elastic Net model using the training data
model_enet.fit(X_train, y_train)

# Predicting target values for the training data using the trained Elastic Net model
pred_train_enet = model_enet.predict(X_train)

# Calculating and printing the root mean squared error (RMSE) for the training predictions
print(np.sqrt(mean_squared_error(y_train, pred_train_enet)))

# Calculating and printing the R-squared score for the training predictions (model's performance on training data)
print(r2_score(y_train, pred_train_enet))

# Predicting target values for the testing data using the trained Elastic Net model
pred_test_enet = model_enet.predict(X_test)

# Calculating and printing the root mean squared error (RMSE) for the testing predictions
print(np.sqrt(mean_squared_error(y_test, pred_test_enet)))

# Calculating and printing the R-squared score for the testing predictions (model's performance on testing data)
print(r2_score(y_test, pred_test_enet))

1664.4184641637066
0.5228653312951645
1671.188594639023
0.5506006141602047


The ElasticNet Regression model is performing the worst. All the other regression models are performing better with a decent R-squared and stable RMSE values. The most ideal result would be an RMSE value of zero and R-squared value of 1, but that's almost impossible in real economic datasets.

There are other iterations that can be done to improve model performance. We have assigned the value of alpha to be 0.01, but this can be altered by hyper parameter tuning to arrive at the optimal alpha value. Cross-validation can also be tried along with feature selection techniques. However, that is not covered in this guide which was aimed at enabling individuals to understand and implement the various Linear Regression models using the scikit-learn library.

In [51]:
# Importing the Linear Regression model
lm = LinearRegression()

# Performing k-fold cross-validation (5 folds in this case)
# - `lm`: The Linear Regression model to evaluate.
# - `X_train, y_train`: The training dataset used for cross-validation.
# - `scoring='r2'`: R-squared is used as the evaluation metric to measure model performance.
# - `cv=5`: Specifies 5-fold cross-validation, meaning the training set is divided into 5 parts:
#   - 4 parts are used for training, and 1 part is used for validation.
#   - This process is repeated 5 times, with each part serving as the validation set once.
scores = cross_val_score(lm, X_train, y_train, scoring='r2', cv=5)

# Displaying the R-squared scores for each fold
scores

array([0.37532808, 0.66029172, 0.49923516, 0.64016747, 0.59784285])

In [58]:
# Initializing a Linear Regression model
lm1 = LinearRegression()

# Performing k-fold cross-validation (5 folds in this case)
# - `lm1`: The Linear Regression model to evaluate.
# - `X_train, y_train`: The training dataset used for cross-validation.
# - `scoring='neg_mean_squared_error'`: The evaluation metric is negative mean squared error (MSE).
#   - Negative MSE is used because cross_val_score expects higher values to indicate better performance.
#   - To interpret the results, take the absolute value of the scores or multiply by -1.
# - `cv=5`: Specifies 5-fold cross-validation (training on 4 folds and validating on the remaining fold, repeated 5 times).
scores = cross_val_score(lm1, X_train, y_train, scoring='neg_mean_squared_error', cv=5)

# Displaying the negative MSE scores for each fold
scores

array([-3154300.02472881, -2324246.90997098, -3041345.78502868,
       -2328392.41592552, -1741550.35886557])

In [57]:
# Importing the sklearn library to access its metrics module
import sklearn

# Retrieving and displaying a sorted list of all available scoring metrics in sklearn
# - `sklearn.metrics.SCORERS.keys()`: Returns a dictionary of scoring metrics that can be used in functions like `cross_val_score` or `GridSearchCV`.
# - `sorted()`: Sorts the list of metric names for easier readability.
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_we

In [59]:
# Step 1: Create a cross-validation scheme
# - `KFold`: Splits the data into `n_splits` folds for cross-validation.
# - `shuffle=True`: Shuffles the data before splitting to ensure randomness.
# - `random_state=100`: Ensures reproducibility of the splits.
folds = KFold(n_splits=5, shuffle=True, random_state=100)

# Step 2: Specify the range of hyperparameters to tune
# - `n_features_to_select`: List of the number of features to be selected by Recursive Feature Elimination (RFE).
# - Here, the range is set to 1 to 3, meaning RFE will try selecting 1, 2, or 3 features.
hyper_params = [{'n_features_to_select': list(range(1, 4))}]

# Step 3: Perform grid search
# 3.1 Specify the base model
# - `lm`: A Linear Regression model is initialized as the base model for RFE.
lm = LinearRegression()

# - `fit`: Fits the Linear Regression model on the training data.
lm.fit(X_train, y_train)

# - `RFE`: Recursive Feature Elimination is initialized with the base model (`lm`).
#   RFE will rank and recursively eliminate features based on their importance.
rfe = RFE(lm)

# 3.2 Initialize GridSearchCV
# - `estimator=rfe`: Specifies RFE as the model for hyperparameter tuning.
# - `param_grid=hyper_params`: The grid of hyperparameters to test (number of features to select).
# - `scoring='r2'`: Uses R-squared as the evaluation metric for model performance.
# - `cv=folds`: Applies the KFold cross-validation scheme.
# - `verbose=1`: Displays progress logs during grid search.
# - `return_train_score=True`: Stores training scores for each fold.
model_cv = GridSearchCV(
    estimator=rfe, 
    param_grid=hyper_params, 
    scoring='r2', 
    cv=folds, 
    verbose=1, 
    return_train_score=True
)

# Fit the GridSearchCV model on the training data
model_cv.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=100, shuffle=True),
             estimator=RFE(estimator=LinearRegression()),
             param_grid=[{'n_features_to_select': [1, 2, 3]}],
             return_train_score=True, scoring='r2', verbose=1)

In [60]:
# cv results/Output
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_features_to_select,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.010243,0.010287,0.000727,0.000901,1,{'n_features_to_select': 1},0.044350,0.382173,-0.027519,0.365837,...,0.141274,0.193000,3,0.032341,0.430245,0.048360,0.434023,0.047866,0.198567,0.190797
1,0.005261,0.003447,0.000361,0.000175,2,{'n_features_to_select': 2},0.118136,0.357447,0.100838,0.381519,...,0.195754,0.145798,2,0.096235,0.499913,0.097406,0.492705,0.111870,0.259626,0.193343
2,0.001317,0.000200,0.000300,0.000075,3,{'n_features_to_select': 3},0.260561,0.384150,0.207754,0.411395,...,0.291270,0.090258,1,0.338543,0.542744,0.353793,0.534538,0.347334,0.423390,0.094262
